In [78]:
import torch.nn as nn
import torch
import torchvision
from torchvision import transforms, datasets, utils
import os
import sys
import json
import time
import torch
import torch.nn as nn
from torchvision import transforms, datasets, utils
import matplotlib.pyplot as plt
import numpy as np
import torch.optim as optim
from tqdm import tqdm
from torchsummaryX import summary
from PIL import Image
from torchvision import transforms
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [79]:
class BaseNet(nn.Module):
    def __init__(self):
        super(BaseNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 5, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 5, 3)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32*5*5, 120)
        self.fc2 = nn.Linear(120, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))    # input(3, 32, 32) output(16, 28, 28)
        x = self.pool1(x)            # output(16, 14, 14)
        x = F.relu(self.conv2(x))    # output(32, 10, 10)
        x = self.pool2(x)            # output(32, 5, 5)
        x = x.view(-1, 32*5*5)       # output(32*5*5)
        x = F.relu(self.fc1(x))      # output(120)
        x = self.fc2(x)              # output(10)
        return x

In [80]:
def train():
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("using {} device.".format(device))

    data_transform = {
        "train": transforms.Compose(
                            [transforms.Resize((224, 224)),
                             transforms.ToTensor()]),
        "val": transforms.Compose(
                        [transforms.Resize((224, 224)),
                         transforms.ToTensor()])}
    data_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))  # get data root path
    image_path = os.path.join(data_root, "data_set", "face_data")  
    assert os.path.exists(image_path), "{} path does not exist.".format(image_path)
    train_dataset = datasets.ImageFolder(root=os.path.join(image_path, "train"),
                                         transform=data_transform["train"])
    train_num = len(train_dataset)


    face_list = train_dataset.class_to_idx
    cla_dict = dict((val, key) for key, val in face_list.items())
    # write dict into json file
    json_str = json.dumps(cla_dict, indent=4)
    with open('class_indices.json', 'w') as json_file:
        json_file.write(json_str)

    batch_size = 32
    nw =0
    print('Using {} dataloader workers every process'.format(nw))

    train_loader = torch.utils.data.DataLoader(train_dataset,
                                               batch_size=batch_size, shuffle=True,
                                               num_workers=nw)

    validate_dataset = datasets.ImageFolder(root=os.path.join(image_path, "val"),
                                            transform=data_transform["val"])
    val_num = len(validate_dataset)
    validate_loader = torch.utils.data.DataLoader(validate_dataset,
                                                  batch_size=4, shuffle=False,
                                                  num_workers=nw)

    print("using {} images for training, {} images for validation.".format(train_num,
                                                                           val_num))


    net = BaseNet()
    xx=torch.zeros(1,3,224,224)
    summary(net,xx)
    net.to(device)
    loss_function = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.0002)

    epochs = 30
    save_path = './BaseNet.pth'
    best_acc = 0.0
    train_steps = len(train_loader)
    timlist=[]
    t_loss=[]
    val_a=[]
    nowtim=time.perf_counter()
    timlist.append(nowtim)
    val_a.append(0)
    t_loss.append(1)
    for epoch in range(epochs):
        # train
        net.train()
        running_loss = 0.0
        train_bar = tqdm(train_loader, file=sys.stdout)
        for step, data in enumerate(train_bar):
            images, labels = data
            optimizer.zero_grad()
            outputs = net(images.to(device))
            loss = loss_function(outputs, labels.to(device))
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()

            train_bar.desc = "train epoch[{}/{}] loss:{:.3f}".format(epoch + 1,
                                                                     epochs,
                                                                     loss)

        # validate
        net.eval()
        acc = 0.0  # accumulate accurate number / epoch
        with torch.no_grad():
            val_bar = tqdm(validate_loader, file=sys.stdout)
            for val_data in val_bar:
                val_images, val_labels = val_data
                outputs = net(val_images.to(device))
                predict_y = torch.max(outputs, dim=1)[1]
                acc += torch.eq(predict_y, val_labels.to(device)).sum().item()

        val_accurate = acc / val_num
        tim=time.perf_counter()
        print('[epoch %d] train_loss: %.3f  val_accuracy: %.3f time:%.3lf' %
              (epoch + 1, running_loss / train_steps, val_accurate,tim-nowtim))
        nowtim=tim
        timlist.append(nowtim)
        val_a.append(val_accurate)
        t_loss.append(running_loss / train_steps)
        if val_accurate > best_acc:
            best_acc = val_accurate
            torch.save(net.state_dict(), save_path)
    print('Finished Training')
    return timlist,val_a,t_loss


In [81]:
def test():
    data_transform = transforms.Compose(
        [transforms.Resize((224, 224)),
         transforms.ToTensor()])

    net = BaseNet()
    net.load_state_dict(torch.load('BaseNet.pth'))


    
    json_path = './class_indices.json'
    assert os.path.exists(json_path), "file: '{}' dose not exist.".format(json_path)

    with open(json_path, "r") as f:
        class_indict = json.load(f)
    path_img = './testimages/'
    ls = os.listdir(path_img)
    pre=[]
    for i in ls:
        img = Image.open(f"./testimages/{i}")

        #plt.imshow(img)
        # [N, C, H, W]
        img = data_transform(img)
        # expand batch dimension
        img = torch.unsqueeze(img, dim=0)
        predict=[]
        with torch.no_grad():
            outputs = net(img)
            predict = torch.max(outputs, dim=1)[1].numpy()
        if predict[0]==1:
            pre.append(1)
        else:
            pre.append(-1)

    return pre

In [82]:
train()

using cpu device.
Using 0 dataloader workers every process
using 2701 images for training, 299 images for validation.
           Kernel Shape     Output Shape   Params Mult-Adds
Layer                                                      
0_conv1   [3, 16, 5, 5]  [1, 16, 74, 74]   1.216k   6.5712M
1_pool1               -  [1, 16, 37, 37]        -         -
2_conv2  [16, 32, 5, 5]  [1, 32, 11, 11]  12.832k   1.5488M
3_pool2               -    [1, 32, 5, 5]        -         -
4_fc1        [800, 120]         [1, 120]   96.12k     96.0k
5_fc2          [120, 2]           [1, 2]    242.0     240.0
------------------------------------------------------------
                        Totals
Total params           110.41k
Trainable params       110.41k
Non-trainable params       0.0
Mult-Adds             8.21624M
100%|██████████████████████████████████████████████████████████████████████████████████| 75/75 [00:01<00:00, 57.38it/s]
[epoch 1] train_loss: 0.442  val_accuracy: 0.936 time:14.489
100%|

In [84]:
pre=test()
path_img = './testimages/'
ls = os.listdir(path_img)
name=[]
for i in ls:
    name.append(i)
import pandas as pd
out_dict = {
    'image_id':list(name),
    'is_male':list(pre)
}
out = pd.DataFrame(out_dict)
out.to_csv('submmit.csv',index=False)